# Example: CitiBike data

Adapted from Kelsey Jordahl
https://gist.github.com/kjordahl/5957573

In [42]:
%matplotlib inline
import requests  
import json
from pyproj import Proj
from shapely.geometry import Point
import geopandas as gp

In [43]:
 
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gp
from shapely.geometry import Point
import pandas as pd

NYC borough boundaries downloaded from [Bytes of the Big Apple](http://www.nyc.gov/html/dcp/download/bytes/nybb_13a.zip)

In [44]:
boros = gp.GeoDataFrame.from_file('nybb_15b/nybb.shp')

Load real time bike station data from [CitiBike](http://citibikenyc.com) json API:

In [45]:
# get the bike data and convert it to a ditcionary
endpoint_url = 'http://citibikenyc.com/stations/json'
response = requests.get(endpoint_url)
data = json.loads(response.text)


In [46]:
# convert the relevant part to a geodataframe
df = gp.GeoDataFrame(data['stationBeanList'])
df.head()

,altitude,availableBikes,availableDocks,city,id,landMark,lastCommunicationTime,latitude,location,longitude,postalCode,stAddress1,stAddress2,stationName,statusKey,statusValue,testStation,totalDocks
0,,19,20,,72,,2017-03-19 05:27:59 PM,40.767272,,-73.993929,,W 52 St & 11 Ave,,W 52 St & 11 Ave,1,In Service,False,39
1,,2,31,,79,,2017-03-19 05:26:19 PM,40.719116,,-74.006667,,Franklin St & W Broadway,,Franklin St & W Broadway,1,In Service,False,33
2,,0,14,,82,,2017-03-19 05:28:56 PM,40.711174,,-74.000165,,St James Pl & Pearl St,,St James Pl & Pearl St,3,Not In Service,False,27
3,,0,0,,83,,2017-03-19 05:28:10 PM,40.683826,,-73.976323,,Atlantic Ave & Fort Greene Pl,,Atlantic Ave & Fort Greene Pl,1,Not In Service,False,62
4,,15,22,,116,,2017-03-19 05:27:26 PM,40.741776,,-74.001497,,W 17 St & 8 Ave,,W 17 St & 8 Ave,1,In Service,False,39


In [97]:
manhattanbike = df['availableBikes']
iin_mn = df.geometry.within(mmanhattanbike)
print(sum(iin_mn), 'bikes are currently available')

AttributeError: No geometry data set yet (expected in column 'geometry'.

In [103]:
df.sum()

altitude                                                                  
availableBikes                                                        3395
availableDocks                                                       12170
city                                                                      
id                                                                 1299113
landMark                                                                  
lastCommunicationTime    2017-03-19 05:27:59 PM2017-03-19 05:26:19 PM20...
latitude                                                           27083.8
location                                                                  
longitude                                                         -49199.2
postalCode                                                                
stAddress1               W 52 St & 11 AveFranklin St & W BroadwaySt Jam...
stAddress2                                                                
stationName              

In [47]:
# there is one row for each bike station.  How many stations are there? 
len(df)

665

In [48]:
boros = gp.read_file('nybb_15b/nybb.shp')
boros

,BoroCode,BoroName,Shape_Area,Shape_Leng,geometry
0,5,Staten Island,1.623827e+09,330466.075042,(POLYGON ((970217.0223999023 145643.3322143555...
1,3,Brooklyn,1.937597e+09,741185.900596,(POLYGON ((1021176.479003906 151374.7969970703...
2,4,Queens,3.045168e+09,897040.298576,(POLYGON ((1029606.076599121 156073.8142089844...
3,1,Manhattan,6.364467e+08,358408.460709,(POLYGON ((981219.0557861328 188655.3157958984...
4,2,Bronx,1.186973e+09,464400.198868,(POLYGON ((1012821.805786133 229228.2645874023...


In [63]:
# in the file above, there are lon-lats, but no geometry field
# we need to set that up

s =gp.GeoSeries([Point(x, y) for x, y in zip(df['longitude'], df['latitude'])])
df['geometry'] = s
df.crs = {'init': 'epsg:4326', 'no_defs': True}
df.geometry.total_bounds

(-74.096936600000006,
 40.661063371900603,
 -73.929891100000006,
 40.804212999999997)

In [64]:
# make sure they are on the same CRS.  
# checking the bounds is a nice way of seeing this
df.to_crs(boros.crs, inplace=True)
df.geometry.total_bounds

(957370.14732175611,
 180120.27032614074,
 1003695.8507545569,
 232275.23054640222)

In [65]:
# the geometry objects can do lots of cool stuff.  For example: 

manhattan = boros.geometry[3]
in_mn = df.geometry.within(manhattan)
print(sum(in_mn), 'stations in Manhattan')

366 stations in Manhattan


# Your turn

You can read about the range of operations available in geopandas here: 
    
http://geopandas.org/index.html

Your assignment is to: 

1. Calculate how many stations are in each borough
2. Calculate how many bikes are currently available in each borough
3. Read about and try at least two new spatial or geometric operations (beyond what I've covered here). 

In [68]:
manhattan = boros.geometry[3]
in_mn = df.geometry.within(manhattan)

Staten_Island = boros.geometry[0]
in_si = df.geometry.within(Staten_Island)

Brooklyn = boros.geometry[1]
in_bk = df.geometry.within(Brooklyn)

Queens = boros.geometry[2]
in_qns = df.geometry.within(Queens)

Bronx = boros.geometry[1]
in_Bronx = df.geometry.within(Bronx )

print(sum(in_mn), 'stations in Manhattan')
print(sum(in_si), 'stations in Staten Island')
print(sum(in_bk), 'stations in Brooklyn')
print(sum(in_qns), 'stations in Brooklyn')
print(sum(in_Bronx), 'stations in Bronx')

366 stations in Manhattan
0 stations in Staten Island
236 stations in Brooklyn
13 stations in Brooklyn
236 stations in Bronx


In [111]:
df.sum()

altitude                                                                  
availableBikes                                                        3395
availableDocks                                                       12170
city                                                                      
id                                                                 1299113
landMark                                                                  
lastCommunicationTime    2017-03-19 05:27:59 PM2017-03-19 05:26:19 PM20...
latitude                                                           27083.8
location                                                                  
longitude                                                         -49199.2
postalCode                                                                
stAddress1               W 52 St & 11 AveFranklin St & W BroadwaySt Jam...
stAddress2                                                                
stationName              